In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import mean_squared_error

# Prepare dataset
def create_sequences(data, window_size):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data[i:i+window_size])
        y.append(data[i+window_size])
    return np.array(X), np.array(y)

def walk_forward_cv(data, window_size=10, n_splits=5, epochs=50):
    fold_size = (len(data) - window_size) // (n_splits + 1)
    errors = []

    for i in range(n_splits):
        end_train = window_size + fold_size * (i + 1)
        train_data = data[:end_train]
        val_data = data[end_train:end_train + fold_size]

        # Skip if not enough validation data
        if len(val_data) < 1:
            break

        # Create sequences
        X_train, y_train = create_sequences(train_data, window_size)
        X_val, y_val = create_sequences(np.concatenate([train_data[-window_size:], val_data]), window_size)

        # Reshape for LSTM
        X_train = X_train[..., np.newaxis]
        X_val = X_val[..., np.newaxis]

        # Build model
        model = tf.keras.Sequential([
            tf.keras.layers.LSTM(128, input_shape=(window_size, 1)),
            tf.keras.layers.Dense(1)
        ])
        model.compile(loss='mse', optimizer='adam')
        model.fit(X_train, y_train, epochs=epochs, verbose=0)

        # Evaluate
        y_pred = model.predict(X_val, verbose=0).flatten()
        error = mean_squared_error(y_val, y_pred)
        errors.append(error)
        print(f"Fold {i+1}/{n_splits}, MSE: {error:.6f}")

    print(f"\nAverage CV MSE: {np.mean(errors):.6f}")
    return errors

# Example usage:
# voltage_data = np.array([...])  # Your 104 measurements
# walk_forward_cv(voltage_data, window_size=10, n_splits=5, epochs=50)
